In [31]:
import random
import pickle
import json
import numpy as np

import nltk
from nltk.stem import WordNetLemmatizer

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD


import warnings
warnings.filterwarnings('ignore')

lemmatizer = WordNetLemmatizer()

In [32]:
data = json.loads(open("C:/Users/kwarg/OneDrive/Documents/codes/Python_Projects/NLP_Projects/ChatBot/intents.json").read())

In [33]:
ignore_chars = [',','.','?','!']
words=[]
documents = []
classes = [] 

In [34]:
for intent in data['intents']:
    for pattern in intent['patterns']:
        word_list = nltk.word_tokenize(pattern)
        words.extend(word_list)
        documents.append((word_list,intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [35]:
words = [lemmatizer.lemmatize(word) for word in words if word not in ignore_chars]
words = sorted(set(words))

classes = sorted(set(classes))

In [36]:
pickle.dump(words, open('./file_words.pkl', 'wb'))
pickle.dump(classes, open('./file_classes.pkl', 'wb'))

In [37]:
training = []
output_empty = [0]*len(classes)

for document in documents:
    bag = []
    word_patterns = document[0]
    word_patterns = [lemmatizer.lemmatize(word.lower()) for word in word_patterns]
    for word in words:
        bag.append(1) if word in word_patterns else bag.append(0)
    output_row = list(output_empty)
    output_row[classes.index(document[1])] = 1
    training.append([bag,output_row])

In [38]:
random.shuffle(training)
training = np.array(training)

train_x = list(training[:,0])
train_y = list(training[:,1])

In [39]:
x_train = list(training[:,0])
y_train = list(training[:,1])

model = Sequential()
model.add(Dense(128, input_shape = (len(x_train[0]),), activation = 'relu' ))
model.add(Dropout(0.5))
model.add(Dense(64, activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(len(y_train[0]), activation = 'softmax'))

sgd = SGD(learning_rate = 0.01)
model.compile(loss = 'categorical_crossentropy', optimizer = 'sgd', metrics = ['accuracy'])

hist = model.fit(np.array(x_train), np.array(y_train), epochs = 3000, batch_size = 8, verbose = 2)

Epoch 1/3000
17/17 - 1s - loss: 3.3428 - accuracy: 0.0385 - 545ms/epoch - 32ms/step
Epoch 2/3000
17/17 - 0s - loss: 3.3414 - accuracy: 0.0154 - 35ms/epoch - 2ms/step
Epoch 3/3000
17/17 - 0s - loss: 3.3425 - accuracy: 0.0538 - 35ms/epoch - 2ms/step
Epoch 4/3000
17/17 - 0s - loss: 3.3389 - accuracy: 0.0462 - 37ms/epoch - 2ms/step
Epoch 5/3000
17/17 - 0s - loss: 3.3331 - accuracy: 0.0538 - 35ms/epoch - 2ms/step
Epoch 6/3000
17/17 - 0s - loss: 3.3361 - accuracy: 0.0462 - 37ms/epoch - 2ms/step
Epoch 7/3000
17/17 - 0s - loss: 3.3237 - accuracy: 0.0692 - 72ms/epoch - 4ms/step
Epoch 8/3000
17/17 - 0s - loss: 3.3135 - accuracy: 0.0692 - 46ms/epoch - 3ms/step
Epoch 9/3000
17/17 - 0s - loss: 3.3276 - accuracy: 0.0769 - 43ms/epoch - 3ms/step
Epoch 10/3000
17/17 - 0s - loss: 3.3195 - accuracy: 0.0615 - 43ms/epoch - 3ms/step
Epoch 11/3000
17/17 - 0s - loss: 3.2873 - accuracy: 0.1308 - 45ms/epoch - 3ms/step
Epoch 12/3000
17/17 - 0s - loss: 3.3108 - accuracy: 0.0692 - 41ms/epoch - 2ms/step
Epoch 13/30

In [40]:
model.save('./chatbot.h5',hist)